In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from scipy.sparse import csr_matrix
from itertools import product
import functools 
import operator 
import regex as re
import time

from classifiers import *
from metrics import *
from kernels import *

from sklearn.model_selection import train_test_split # lui il va partir mais pour l'instant c'est pratique


# Load the data

In [2]:
from csv import reader

def features_into_array(path):
    with open(path, 'r') as read_obj:
        csv_reader = reader(read_obj)
        header = next(csv_reader)
        X = list()
        if header != None:
            for row in csv_reader:
                # row variable is a list that represents a row in csv
                X.append(np.array(row[1]))
                
    X = np.array(X) ## dtype might be changed in something more convenient. For now, dtype = "<U1"
    return X

In [3]:
Xtr0 = features_into_array("data/Xtr0.csv")
Ytr0 = np.genfromtxt("data/Ytr0.csv", delimiter=',', skip_header=1)

Xtr1 = features_into_array("data/Xtr1.csv")
Ytr1 = np.genfromtxt("data/Ytr1.csv", delimiter=',', skip_header=1)

Xtr2 = features_into_array("data/Xtr2.csv")
Ytr2 = np.genfromtxt("data/Ytr2.csv", delimiter=',', skip_header=1)

In [4]:
Xtr0[0]

'TCCTGTGCACATCTGCACCCCTGTTGTGGCCACAAAATGATCCGGCACCACCCAGTGGGAGACGACAGAGGTGGCAATGGGGTGTCGGCTCTGACGCCTCC'

## Spectrum kernel

For a fixed value k (that needs to be tuned), the k-spectrum kernel is defined as : 


\begin{align*}
K(x,x^{\prime}) := \sum_{u \in \mathcal{A}^k} \phi_{u}(x) \phi_{u}(x^{\prime})
\end{align*}

In [5]:
def all_possible_substrings(k):
    """
    With a k spectrum kernel, let us find all the possible combinations of chars of size k in the sequence x
    This way, we could index them in the sequence x
    """
    char_list = list(['A', 'C','G','T'])
    alphabet_tuples = list(product(char_list,repeat=k))
    alphabet = dict()
    idx=0
    for i in alphabet_tuples:
        alphabet[functools.reduce(operator.add, (i))] = idx
        idx += 1
        #alphabet.append(functools.reduce(operator.add, (i)))
    return alphabet

In [6]:
dict6 = all_possible_substrings(3)

In [7]:
def pre_indexing(X, k, alphabet=None):
    """
    Outputs a sparse matrix of shape Transforms an input array into a sparse matrix encoding the number of occurences of each letter of
    the alphabet composed of substrings of size k
    """
    i = 0
    n = X.shape[0]
    if alphabet is None:
        alphabet = all_possible_substrings(k)
    D = np.zeros((n,len(alphabet)))
    
    for i in range(X.shape[0]):
        idx=0
        while idx + k < len(X[i]):
            D[i, alphabet[X[i][idx:idx+k]]] += 1
            idx += 1
    """
    for x in X:
        d = dict((letter, len(re.findall(letter, x, overlapped=True))) 
                             for letter in alphabet)
        data = np.array(list(d.items()))
        D[i] = data[:,1]
        i+=1
    """
    D = csr_matrix(D, dtype = int)
    return D

In [8]:
k = 6
start_time = time.time()
alphabet_6 = all_possible_substrings(k)
mm = pre_indexing(Xtr0, 6, alphabet=alphabet_6)
print("--- Found alphabet in %s seconds ---" % (time.time() - start_time))


--- Found alphabet in 1.7300939559936523 seconds ---


In [9]:
def spectrum_kernel(X_train, X_val, X_test, k, alphabet=None):
    # Kill two birds with one stone and compute K_train, K_val and K_test all at once.
    """
    Computes the spectrum kernels for X_train (n_train x n_train), X_validation and X_test
    (on the RKHS generated by X_train's samples) which is of shape n_validation x n_train (resp n_test x n_train)
    """
    if alphabet is None:
        #D_train = pre_indexing(X_train,k).toarray()
        #D_val = pre_indexing(X_val,k).toarray()
        alphabet = all_possible_substrings(k)
   
    D_train = pre_indexing(X_train,k,alphabet)
    D_val = pre_indexing(X_val,k,alphabet)
    D_test = pre_indexing(X_test,k,alphabet)
        
        
    #K_val = np.inner(D_val, D_train)
    #K_val = K_val.astype('float')
    
    K_train = D_train.dot(D_train.transpose())
    K_train = K_train.toarray().astype('float')
    
    K_val = D_val.dot(D_train.transpose())
    K_val = K_val.toarray().astype('float')
    
    K_test = D_test.dot(D_train.transpose())
    K_test = K_test.toarray().astype('float')
    
        
    return(K_train, K_val, K_test)

# Compute the spectrum-kernels for our data

In [11]:
Xtr0_, Xval0_, ytr0, yval0 = train_test_split(Xtr0, Ytr0, test_size=0.5, random_state=42)
Xtr1_, Xval1_, ytr1, yval1 = train_test_split(Xtr1, Ytr1, test_size=0.5, random_state=42)
Xtr2_, Xval2_, ytr2, yval2 = train_test_split(Xtr2, Ytr2, test_size=0.5, random_state=42)

Xte0 = features_into_array("data/Xte0.csv")
Xte1 = features_into_array("data/Xte1.csv")
Xte2 = features_into_array("data/Xte2.csv")

In [12]:
k = 8
start_time = time.time()
alphabet_8 = all_possible_substrings(k)
print("--- Found alphabet in %s seconds ---" % (time.time() - start_time))
start_time = time.time()

K_tr0, K_val0, K_te0 = spectrum_kernel(Xtr0_, Xval0_, Xte0, k, alphabet=alphabet_8,)
print("--- Computed kernel in %s seconds ---" % (time.time() - start_time))
start_time = time.time()
K_tr1, K_val1, K_te1 = spectrum_kernel(Xtr1_, Xval1_, Xte1, k, alphabet=alphabet_8)
print("--- Computed kernel in %s seconds ---" % (time.time() - start_time))
start_time = time.time()
K_tr2, K_val2, K_te2 = spectrum_kernel(Xtr2_, Xval2_, Xte2, k, alphabet=alphabet_8)
print("--- Computed kernel in %s seconds ---" % (time.time() - start_time))

--- Found alphabet in 0.19686412811279297 seconds ---
--- Computed kernel in 9.518135070800781 seconds ---
--- Computed kernel in 9.735282182693481 seconds ---
--- Computed kernel in 10.768494129180908 seconds ---


# Training-Validation
(runs to make sure everything is ok)

In [13]:
lambdas = [0] + [10**i for i in range(-10,2)]

print("************* KRR for dataset 0*************\n")
alphas_tr0, loss_tr0, acc_0, loss_val0, acc_val0 = KRR(K_tr0, ytr0[:,1], K_val0, yval0[:,1], lambdas)
print("")
print("")
print("************* KRR for dataset 1 *************\n")
alphas_tr1, loss_tr1, acc_1, loss_val1, acc_val1 = KRR(K_tr1, ytr1[:,1], K_val1, yval1[:,1],lambdas)

print("")
print("")
print("************* KRR for dataset 2 *************\n")
alphas_tr2, loss_tr2, acc_2, loss_val2, acc_val2 = KRR(K_tr2, ytr2[:,1], K_val2, yval2[:,1],lambdas)


************* KRR for dataset 0*************

***********lambda = 0***********
Training: loss = 500.9277, accuracy = 1.000000
Validation: loss = 305.9762, accuracy = 0.553000
***********lambda = 1e-10***********
Training: loss = 500.9277, accuracy = 1.000000
Validation: loss = 305.9762, accuracy = 0.553000
***********lambda = 1e-09***********
Training: loss = 500.9277, accuracy = 1.000000
Validation: loss = 305.9762, accuracy = 0.553000
***********lambda = 1e-08***********
Training: loss = 500.9276, accuracy = 1.000000
Validation: loss = 305.9762, accuracy = 0.553000
***********lambda = 1e-07***********
Training: loss = 500.9271, accuracy = 1.000000
Validation: loss = 305.9762, accuracy = 0.553000
***********lambda = 1e-06***********
Training: loss = 500.9222, accuracy = 1.000000
Validation: loss = 305.9760, accuracy = 0.553000
***********lambda = 1e-05***********
Training: loss = 500.8728, accuracy = 1.000000
Validation: loss = 305.9738, accuracy = 0.553000
***********lambda = 0.0001*

In [14]:
lambdas = [10**i for i in range(-4,1)]

print("*************KLR for dataset 0*************\n")
alphas_tr0, loss_tr0, acc_0, loss_val0, acc_val0 = KLR(K_tr0, ytr0[:,1], K_val0, yval0[:,1], lambdas, tresh=1e-8)
print("")
print("")
print("************* KLR for dataset 1 *************\n")
alphas_tr1, loss_tr1, acc_1, loss_val1, acc_val1 = KLR(K_tr1, ytr1[:,1], K_val1, yval1[:,1],lambdas, tresh=1e-8)

print("")
print("")
print("************* KLR for dataset 2 *************\n")
alphas_tr2, loss_tr2, acc_2, loss_val2, acc_val2 = KLR(K_tr2, ytr2[:,1], K_val2, yval2[:,1],lambdas, tresh= 1e-8)


*************KLR for dataset 0*************

***********lambda = 0.0001***********
Training: loss = 0.3147, accuracy = 1.000000
Validation: loss = 0.6500, accuracy = 0.616000
***********lambda = 0.001***********
Training: loss = 0.3267, accuracy = 1.000000
Validation: loss = 0.6498, accuracy = 0.619000
***********lambda = 0.01***********
Training: loss = 0.4057, accuracy = 1.000000
Validation: loss = 0.6512, accuracy = 0.625000
***********lambda = 0.1***********
Training: loss = 0.5857, accuracy = 0.980000
Validation: loss = 0.6649, accuracy = 0.631000
***********lambda = 1***********
Training: loss = 0.6745, accuracy = 0.967000
Validation: loss = 0.6852, accuracy = 0.605000


************* SVM for dataset 1 *************

***********lambda = 0.0001***********
Training: loss = 0.3147, accuracy = 1.000000
Validation: loss = 0.6621, accuracy = 0.600000
***********lambda = 0.001***********
Training: loss = 0.3270, accuracy = 1.000000
Validation: loss = 0.6626, accuracy = 0.603000
********

In [15]:
lambdas = [10**i for i in range(-4, 3)]
print("************* SVM for dataset 0*************\n")
alphas_tr0, loss_tr0, acc_0, loss_val0, acc_val0 = SVM(K_tr0, ytr0[:,1], K_val0, yval0[:,1], lambdas)
print("")
print("")
print("************* SVM for dataset 1 *************\n")
alphas_tr1, loss_tr1, acc_1, loss_val1, acc_val1 = SVM(K_tr1, ytr1[:,1], K_val1, yval1[:,1],lambdas)

print("")
print("")
print("************* SVM for dataset 2 *************\n")
alphas_tr2, loss_tr2, acc_2, loss_val2, acc_val2 = SVM(K_tr2, ytr2[:,1], K_val2, yval2[:,1],lambdas)


************* SVM for dataset 0*************

---------------  lambda = 0.0001  ---------------
Training: loss = 0.982613, accuracy = 0.955000
Validation: loss = 0.991697, accuracy = 0.598000
---------------  lambda = 0.001  ---------------
Training: loss = 0.837311, accuracy = 0.962000
Validation: loss = 0.925864, accuracy = 0.601000
---------------  lambda = 0.01  ---------------
Training: loss = 0.117413, accuracy = 0.988000
Validation: loss = 0.854595, accuracy = 0.635000
---------------  lambda = 0.1  ---------------
Training: loss = 0.000000, accuracy = 1.000000
Validation: loss = 0.879713, accuracy = 0.611000
---------------  lambda = 1  ---------------
Training: loss = 0.000000, accuracy = 1.000000
Validation: loss = 0.879708, accuracy = 0.611000
---------------  lambda = 10  ---------------
Training: loss = 0.000000, accuracy = 1.000000
Validation: loss = 0.879703, accuracy = 0.611000
---------------  lambda = 100  ---------------
Training: loss = 0.000000, accuracy = 1.000000

## Gridsearch on SVM

In [16]:
from sklearn.model_selection import GridSearchCV

In [17]:
# sanity check to see that our SVM does a good job
# C = 1/(2*n*lambda) 
from sklearn import svm
clf = svm.SVC(kernel='precomputed', C =1.)
print("*"*15  + "Sklearn SVM on dataset 0 " + "*"*15)
clf.fit(K_tr0, ytr0[:,1])
print(clf.score(K_tr0, ytr0[:,1]))
print(clf.score(K_val0, yval0[:,1]))
lambd_C = [1]
alphas_tr0, loss_tr0, acc_0, loss_val0, acc_val0 = SVM(K_tr0, ytr0[:,1], K_val0, yval0[:,1], lambd_C)

print("")
print("")
print("")

print("*"*15  + "Sklearn SVM on dataset 1 " + "*"*15)
clf.fit(K_tr1, ytr1[:,1])
print(clf.score(K_tr1, ytr1[:,1]))
print(clf.score(K_val1, yval1[:,1]))
lambd_C = [1]
alphas_tr1, loss_tr1, acc_1, loss_val1, acc_val1 = SVM(K_tr1, ytr1[:,1], K_val1, yval1[:,1], lambd_C)

print("")
print("")
print("")

print("*"*15  + "Sklearn SVM on dataset 2 " + "*"*15)
clf.fit(K_tr2, ytr2[:,1])
print(clf.score(K_tr2, ytr2[:,1]))
print(clf.score(K_val2, yval2[:,1]))
lambd_C = [1]
alphas_tr2, loss_tr2, acc_2, loss_val2, acc_val2 = SVM(K_tr2, ytr2[:,1], K_val2, yval2[:,1], lambd_C)


***************Sklearn SVM on dataset 0 ***************
1.0
0.61
---------------  lambda = 1  ---------------
Training: loss = 0.000000, accuracy = 1.000000
Validation: loss = 0.879708, accuracy = 0.611000



***************Sklearn SVM on dataset 1 ***************
1.0
0.603
---------------  lambda = 1  ---------------
Training: loss = 0.000000, accuracy = 1.000000
Validation: loss = 0.915497, accuracy = 0.604000



***************Sklearn SVM on dataset 2 ***************
1.0
0.703
---------------  lambda = 1  ---------------
Training: loss = 0.000000, accuracy = 1.000000
Validation: loss = 0.753376, accuracy = 0.702000


### Okay cool almost there let's do gridsearch

#### For dataset 1

In [19]:
Kernels_tr_0 = []
Kernels_val_0 = []
Kernels_te_0 = []

Kernels_tr_1 = []
Kernels_val_1 = []
Kernels_te_1 = []

Kernels_tr_2 = []
Kernels_val_2 = []
Kernels_te_2 = []


for k in range(3,11):
    
    print("*"*15 + "treating k=" + str(k) + 15*"*")
    start_time = time.time()
    alphabet_k = all_possible_substrings(k)
    print("--- Found alphabet in %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    K_tr0, K_val0, K_te0 = spectrum_kernel(Xtr0_, Xval0_, Xte0, k, alphabet=alphabet_k)
    Kernels_tr_0 += [K_tr0]
    Kernels_val_0 += [K_val0]
    Kernels_te_0 += [K_te0]
    
    K_tr1, K_val1, K_te1 = spectrum_kernel(Xtr1_, Xval1_, Xte1, k, alphabet=alphabet_k)
    Kernels_tr_1 += [K_tr1]
    Kernels_val_1 += [K_val1]
    Kernels_te_1 += [K_te1]
    
    K_tr2, K_val2, K_te2 = spectrum_kernel(Xtr2_, Xval2_, Xte2, k, alphabet=alphabet_k)
    Kernels_tr_2 += [K_tr2]
    Kernels_val_2 += [K_val2]
    Kernels_te_2 += [K_te2]
    
    print("--- Computed all the kernels in %s seconds ---" % (time.time() - start_time))
    print("")
    print("")
    print("")
    

***************treating k=3***************
--- Found alphabet in 0.00162506103515625 seconds ---
--- Computed all the kernels in 8.833793878555298 seconds ---



***************treating k=4***************
--- Found alphabet in 0.0005450248718261719 seconds ---
--- Computed all the kernels in 8.055458068847656 seconds ---



***************treating k=5***************
--- Found alphabet in 0.001964092254638672 seconds ---
--- Computed all the kernels in 8.84161376953125 seconds ---



***************treating k=6***************
--- Found alphabet in 0.01622772216796875 seconds ---
--- Computed all the kernels in 9.374190092086792 seconds ---



***************treating k=7***************
--- Found alphabet in 0.05209493637084961 seconds ---
--- Computed all the kernels in 10.316075086593628 seconds ---



***************treating k=8***************
--- Found alphabet in 0.23636889457702637 seconds ---
--- Computed all the kernels in 27.527786016464233 seconds ---



***************treating 

In [20]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV

values_C = [j*10**i for i in range(-5,3) for j in range(1,10)]
parameters = {'C': values_C}
svm__ = svm.SVC(kernel='precomputed')
gs_k = GridSearchCV(svm__, param_grid=parameters, refit=True, verbose=0)

for k in range(2,8):
    
        
    print("*"*15 + "treating k = " + str(k+3) + 15*"*")
    print("")
    print("-"*15 + " treating dataset 0 "+ 15*"-")
    
    gs_k.fit(Kernels_tr_0[k], ytr0[:,1])
    print(gs_k.best_estimator_)
    print(f"training score for k = {k+3} ", gs_k.score(Kernels_tr_0[k], ytr0[:,1]))
    print(f"validation score for k = {k+3} ", gs_k.score(Kernels_val_0[k], yval0[:,1]))
    
    print("")
    print("-"*15 + " treating dataset 1 "+ 15*"-")
    
    gs_k.fit(Kernels_tr_1[k], ytr1[:,1])
    print(gs_k.best_estimator_)
    print(f"training score for k = {k+3} ", gs_k.score(Kernels_tr_1[k], ytr1[:,1]))
    print(f"validation score for k = {k+3} ", gs_k.score(Kernels_val_1[k], yval1[:,1]))
    
    print("")
    print("-"*15 + " treating dataset 2 "+ 15*"-")
    
    gs_k.fit(Kernels_tr_2[k], ytr2[:,1])
    print(gs_k.best_estimator_)
    print(f"training score for k = {k+3} ", gs_k.score(Kernels_tr_2[k], ytr2[:,1]))
    print(f"validation score for k = {k+3} ", gs_k.score(Kernels_val_2[k], yval2[:,1]))
    
    print("")
    print("")

***************treating k = 5***************

--------------- treating dataset 0 ---------------
SVC(C=0.004, kernel='precomputed')
training score for k = 5  0.778
validation score for k = 5  0.598

--------------- treating dataset 1 ---------------
SVC(C=0.007, kernel='precomputed')
training score for k = 5  0.832
validation score for k = 5  0.623

--------------- treating dataset 2 ---------------
SVC(C=0.007, kernel='precomputed')
training score for k = 5  0.859
validation score for k = 5  0.7


***************treating k = 6***************

--------------- treating dataset 0 ---------------
SVC(C=0.009000000000000001, kernel='precomputed')
training score for k = 6  0.936
validation score for k = 6  0.602

--------------- treating dataset 1 ---------------
SVC(C=0.004, kernel='precomputed')
training score for k = 6  0.892
validation score for k = 6  0.611

--------------- treating dataset 2 ---------------
SVC(C=0.007, kernel='precomputed')
training score for k = 6  0.93
validation s

# fuck it lets do for k = 11 and 12 too
for k in range(11,13):
    print("*"*15 + "treating k=" + str(k) + 15*"*")
    start_time = time.time()
    alphabet_k = all_possible_substrings(k)
    print("--- Found alphabet in %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    K_tr0, K_val0, K_te0 = spectrum_kernel(Xtr0_, Xval0_, Xte0, k, alphabet=alphabet_k)
    Kernels_tr_0 += [K_tr0]
    Kernels_val_0 += [K_val0]
    Kernels_te_0 += [K_te0]
    
    K_tr1, K_val1, K_te1 = spectrum_kernel(Xtr1_, Xval1_, Xte1, k, alphabet=alphabet_k)
    Kernels_tr_1 += [K_tr1]
    Kernels_val_1 += [K_val1]
    Kernels_te_1 += [K_te1]
    
    K_tr2, K_val2, K_te2 = spectrum_kernel(Xtr2_, Xval2_, Xte2, k, alphabet=alphabet_k)
    Kernels_tr_2 += [K_tr2]
    Kernels_val_2 += [K_val2]
    Kernels_te_2 += [K_te2]
    
    print("--- Computed all the kernel in %s seconds ---" % (time.time() - start_time))
    print("")
    print("")
    print("")
    

forget it im scared

# Save the best model

In [ ]:
test_kernels = [K_te0, K_te1, K_te2]
#test_alphas = [alphas_tr0[-4], alphas_tr1[-4], alphas_tr2[-3]] # il faut choisir l'alpha associé à un bon lambda!
test_alphas = [alphas_tr0[0], alphas_tr1[0], alphas_tr2[0]]
write_predictions_csv(test_kernels, test_alphas, path ="data/Ytest_sequences.csv", mode="SVM")

In [ ]:
K_te0.shape

In [ ]:
K_tr0[0]

In [ ]:
K_te0[0,0:4]